# FancyPuzzle
## 概要
このノートブックでは、長方形に限らず、好きな形のパズルを生成する方法を考えます。

In [2]:
import os
import sys
import copy
import datetime
import time
import math
import itertools
import unicodedata
import collections
import pickle
import shutil

import numpy as np
import pandas as pd
from PIL import Image
from IPython.display import display, HTML
import matplotlib.pyplot as plt

sys.path.append('../python')
from pyzzle import Puzzle, Dictionary, Placeable, ObjectiveFunction, Optimizer
from src import utils

## FancyPuzzle

In [3]:
class FancyPuzzle(Puzzle):
    def __init__(self, width, height, mask=None, title="スケルトンパズル", msg=True):
        if mask is None:
            mask = np.ones(width*height, dtype="bool").reshape(height, width)
        self.mask = mask
        
        super().__init__(width, height, title, msg)

    def isEnabledAdd(self, div, i, j, word, wLen):
        """
        This method determines if a word can be placed
        """
        if div == 0:
            if np.any(self.mask[i:i+wLen, j] == False):
                return 7
        if div == 1:
            if np.any(self.mask[i, j:j+wLen] == False):
                return 7
    
        return super().isEnabledAdd(div, i, j, word, wLen)

    def saveImage(self, data, fpath, list_label="[Word List]", dpi=100):
        """
        This method generates and returns a puzzle image with a word list
        """
        # Generate puzzle image
        colors = np.where(self.cover<1, "#000000", "#FFFFFF")
        df = pd.DataFrame(data)

        fig=plt.figure(figsize=(16, 8), dpi=dpi)
        ax1=fig.add_subplot(121) # puzzle
        ax2=fig.add_subplot(122) # word list
        ax1.axis("off")
        ax2.axis("off")
        fig.set_facecolor('#EEEEEE')
        # Draw puzzle
        ax1_table = ax1.table(cellText=df.values, cellColours=colors, cellLoc="center", bbox=[0, 0, 1, 1], fontsize=20)
#         for _, cell in ax1_table.get_celld().items():
#             cell.set_text_props(size=20)
        ax1.set_title(label=f"*** {self.title} ***", size=20)
        
        # delete unmasked cells
        mask = np.where(puzzle.mask == False)
        for i, j in list(zip(mask[0], mask[1])):
            del ax1_table._cells[i, j]

        # Draw word list
        words = [word for word in self.usedWords if word != ""]
        if words == []:
            words = [""]
        words.sort()
        words = sorted(words, key=len)

        rows = self.height
        cols = math.ceil(len(words)/rows)
        padnum = cols*rows - len(words)
        words += ['']*padnum
        words = np.array(words).reshape(cols, rows).T

        ax2_table = ax2.table(cellText=words, cellColours=None, cellLoc="left", edges="open", bbox=[0, 0, 1, 1])
        ax2.set_title(label=list_label, size=20)
        for _, cell in ax2_table.get_celld().items():
            cell.set_text_props(size=18)
        plt.tight_layout()
        plt.savefig(fpath, dpi=dpi)
        plt.close()
    
    def jump(self, idx):
        tmp_puzzle = self.__class__(self.width, self.height, self.mask, self.title, msg=False)
        tmp_puzzle.dic = copy.deepcopy(self.dic)
        tmp_puzzle.plc = Placeable(self.width, self.height, tmp_puzzle.dic, msg=False)
        tmp_puzzle.optimizer = copy.deepcopy(self.optimizer)
        tmp_puzzle.objFunc = copy.deepcopy(self.objFunc)
        tmp_puzzle.baseHistory = copy.deepcopy(self.baseHistory)
        
        if set(self.history).issubset(self.baseHistory) is False:
            if idx <= len(self.history):
                tmp_puzzle.baseHistory = copy.deepcopy(self.history)
            else:
                raise RuntimeError('This puzzle is up to date')

        for code, k, div, i, j in tmp_puzzle.baseHistory[:idx]:
            if code == 1:
                tmp_puzzle._add(div, i, j, k)
            elif code in (2,3):
                tmp_puzzle._drop(div, i, j, k)
        tmp_puzzle.initSol = True
        return tmp_puzzle

    def move(self, direction, n=0, limit=False):
        super().move(direction, n, limit)

### フォント設定
本ライブラリにおける画像化には`matplotlib`が用いられますが、`matplotlib`はデフォルトで日本語に対応したフォントを使わないので、`rcParams`を用いてデフォルトのフォント設定を変更します。

In [4]:
# font setting
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Hiragino Maru Gothic Pro', 'Yu Gothic', 'Meirio', 'Takao', 'IPAexGothic', 'IPAPGothic', 'Noto Sans CJK JP']

## 実行

In [5]:
fpath = "../dict/pokemon.txt"  # countries hokkaido animals kotowaza birds dinosaurs fishes sports pokemon typhoon
width = 15
height = 15
seed = 2
withWeight = False

np.random.seed(seed=seed)
start = time.time()

In [6]:
mask = np.array([
    [0,0,0,0,0,1,1,1,1,1,0,0,0,0,0],
    [0,0,0,1,1,1,1,1,1,1,1,1,0,0,0],
    [0,0,1,1,1,1,1,1,1,1,1,1,1,0,0],
    [0,1,1,1,1,1,1,1,1,1,1,1,1,1,0],
    [0,1,1,1,1,1,1,1,1,1,1,1,1,1,0],
    [1,1,1,1,1,1,0,0,0,1,1,1,1,1,1],
    [1,1,1,1,1,0,0,0,0,0,1,1,1,1,1],
    [1,1,1,1,1,0,0,0,0,0,1,1,1,1,1],
    [1,1,1,1,1,0,0,0,0,0,1,1,1,1,1],
    [1,1,1,1,1,1,0,0,0,1,1,1,1,1,1],
    [0,1,1,1,1,1,1,1,1,1,1,1,1,1,0],
    [0,1,1,1,1,1,1,1,1,1,1,1,1,1,0],
    [0,0,1,1,1,1,1,1,1,1,1,1,1,0,0],
    [0,0,0,1,1,1,1,1,1,1,1,1,0,0,0],
    [0,0,0,0,0,1,1,1,1,1,0,0,0,0,0],
], dtype="bool")
# Make instances
puzzle = FancyPuzzle(width, height, mask, "ドーナツパズル")
dic = Dictionary(fpath)
if not withWeight:
    dic.calcWeight()
objFunc = ObjectiveFunction()
optimizer = Optimizer()

puzzle.importDict(dic)

FancyPuzzle object has made.
 - title       : ドーナツパズル
 - width       : 15
 - height      : 15
 - cell' shape : (width, height) = (15,15)
Dictionary object has made.
 - file path         : ../dict/pokemon.txt
 - dictionary size   : 809
 - top of dictionary : {'word': 'フシギダネ', 'weight': 0, 'len': 5}
All weights are calculated.
TOP 5 characters:
[('ー', 267), ('ン', 219), ('ル', 180), ('ラ', 134), ('ス', 113)]
TOP 5 words:
['ランターン' 'ブーバーン' 'マーシャドー' 'オンバーン' 'アンノーン']
ObjectiveFunction object has made.
Optimizer object has made.
Imported Dictionary name: `pokemon`, size: 809
Placeable size : 276900


In [7]:
# Register and set method and compile
objFunc.register(["totalWeight", "solSize", "crossCount", "fillCount", "maxConnectedEmpties"])
optimizer.setMethod("localSearch")
puzzle.compile(objFunc=objFunc, optimizer=optimizer)

 - 'totalWeight' function has registered.
 - 'solSize' function has registered.
 - 'crossCount' function has registered.
 - 'fillCount' function has registered.
 - 'maxConnectedEmpties' function has registered.
 - 'localSearch' method has registered.
compile succeeded.
 --- objective functions:
  |-> 0 totalWeight
  |-> 1 solSize
  |-> 2 crossCount
  |-> 3 fillCount
  |-> 4 maxConnectedEmpties
 --- optimizer: localSearch


In [8]:
# Solve
puzzle.firstSolve()
puzzle.solve(epoch=10)
print(f"SimpleSolution: {puzzle.isSimpleSol()}")
puzzle.saveAnswerImage(f"fig/puzzle/{dic.name}_w{width}_h{height}_r{seed}.png", "【単語リスト】")

>>> Interim solution


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,,,,,,,,,,,,,,,
1,,,,,フ,,メ,レ,シ,ー,,,,,
2,,,,ポ,カ,ブ,,シ,,,ロ,,,,
3,,ヨ,,,マ,,,ラ,ッ,キ,ー,,ヨ,,
4,,ワ,,マ,ル,ノ,ー,ム,,,ブ,,ノ,,
5,ア,シ,マ,リ,,,,,,,シ,ャ,ワ,ー,ズ
6,,,,ル,,,,,,,ン,,ー,,ル
7,,キ,モ,リ,,,,,,,,,ル,,ズ
8,,ュ,,,デ,,,,,,,,,,キ
9,,ウ,ツ,ド,ン,,,,,,ア,ー,ゴ,ヨ,ン


>>> Epoch 1/10
    - Stayed: [9886   24   23   80  159]
>>> Epoch 2/10
    - Stayed: [9886   24   23   80  159]
>>> Epoch 3/10
    - Stayed: [9886   24   23   80  159]
>>> Epoch 4/10
    - Stayed: [9886   24   23   80  159]
>>> Epoch 5/10
    - Stayed: [9886   24   23   80  159]
>>> Epoch 6/10
    - Stayed: [9886   24   23   80  159]
>>> Epoch 7/10
    - Improved: [9886   24   23   80  159] --> [11059    24    23    80   168]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,,,,,,,,,,,,,,,
1,,,,,フ,,メ,レ,シ,ー,,,,,
2,,,,ポ,カ,ブ,,シ,,,ロ,,,,
3,,ヨ,,,マ,,,ラ,ッ,キ,ー,,ヨ,,
4,,ワ,,マ,ル,ノ,ー,ム,,,ブ,,ノ,,
5,ア,シ,マ,リ,,,,,,,シ,ャ,ワ,ー,ズ
6,,,,ル,,,,,,,ン,,ー,,ル
7,,キ,モ,リ,,,,,,,,,ル,,ズ
8,,ン,,,,,,,,,,,,,キ
9,,グ,,,ド,,,,,,ア,ー,ゴ,ヨ,ン


>>> Epoch 8/10
    - Improved: [11059    24    23    80   168] --> [11148    25    24    80   175]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,,,,,,,,,,ド,,,,,
1,,,,,フ,,メ,レ,シ,ー,,,,,
2,,,,ポ,カ,ブ,,シ,,タ,,ル,,,
3,,ヨ,,,マ,,,ラ,,ク,ヌ,ギ,ダ,マ,
4,,ワ,,マ,ル,ノ,ー,ム,,ン,,ア,,シ,
5,ア,シ,マ,リ,,,,,,,,,,ェ,
6,,,,ル,,,,,,,ハ,リ,ボ,ー,グ
7,,キ,モ,リ,,,,,,,,,,ド,
8,,ン,,,,,,,,,サ,ン,ド,,
9,,グ,,,ド,,,,,,イ,,,ア,


>>> Epoch 9/10
    - Stayed: [11148    25    24    80   175]
>>> Epoch 10/10
    - Stayed: [11148    25    24    80   175]
 --- done
SimpleSolution: True


In [9]:
e_time = time.time() - start
print (f"e_time: {format(e_time)} s")

e_time: 86.03579688072205 s
